# 감성분석

## 1. 지도학습

### 실습: IMDB 영화 감상평 감성 분석

In [39]:
import pandas as pd
import numpy as np
import re

#### kaggle IMDB 영화감상평 데이터 불러오기

In [49]:
review_df = pd.read_csv('./필요 데이터/labeledTrainData.tsv', sep = '\t')
review_df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [50]:
review_df['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [51]:
re.sub(r'<\w+ />', ' ', review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.  Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.  The actual feature film bit when it finally starts is only on for 2

#### 텍스트 정규화

In [52]:
print(review_df['review'][0])

review_df['review'] = review_df['review'].str.replace('<br />', ' ')
review_df['review'] = review_df['review'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

review_df['review'][0]

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally star

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

#### 학습용/테스트용 데이터 분리

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
train_df = review_df['review']
test_df = review_df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(train_df, test_df, 
                                                    test_size = 0.3,
                                                    random_state = 156)

X_train.shape, X_test.shape

((17500,), (7500,))

#### 하이퍼파라미터 튜닝 및 파이프라인을 활용한 모델링

In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [63]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression(solver = 'liblinear'))
])

params = {
    'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
    'tfidf_vect__max_df': [1.0, 0.8],
    'tfidf_vect__min_df': [1, 10, 20],
    'tfidf_vect__max_features': [None, 0.8],
}

grid_clf = GridSearchCV(pipeline, param_grid = params, cv = 3, 
                        scoring = 'accuracy', verbose = 1)

# 학습
grid_clf.fit(X_train, y_train)

print(f'최적의 파라미터: {grid_clf.best_params_}')

# 예측
pred = grid_clf.predict(X_test)


# 평가
accuracy = accuracy_score(y_test, pred)
print(f'정확도: {accuracy:.4f}')

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\NEW\anaconda3\envs\cakd7\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Use

최적의 파라미터: {'tfidf_vect__max_df': 1.0, 'tfidf_vect__max_features': None, 'tfidf_vect__min_df': 1, 'tfidf_vect__ngram_range': (1, 1)}
정확도: 0.8853


In [68]:
# 예측
pred_proba = grid_clf.predict_proba(X_test)[:, 1]


# 평가
auc = roc_auc_score(y_test, pred_proba)
print(f'auc: {auc:.4f}')

auc: 0.9547


In [69]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression(solver = 'liblinear'))
])

params = {
    'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
    'tfidf_vect__max_df': [1.0, 0.8],
    'tfidf_vect__min_df': [1, 10, 20],
    'tfidf_vect__max_features': [None, 0.8],
}

grid_clf = GridSearchCV(pipeline, param_grid = params, cv = 3, 
                        scoring = 'roc_auc', verbose = 1)

# 학습
grid_clf.fit(X_train, y_train)

print(f'최적의 파라미터: {grid_clf.best_params_}')

# 예측
pred = grid_clf.predict(X_test)


# 평가
accuracy = accuracy_score(y_test, pred)
print(f'정확도: {accuracy:.4f}')

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 330, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\NEW\AppData\Roaming\Python\Python38\site-packages\sklearn\pipeline.py", line 292, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "C:\Users\NEW\anaconda3\envs\cakd7\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "C:\Use

최적의 파라미터: {'tfidf_vect__max_df': 1.0, 'tfidf_vect__max_features': None, 'tfidf_vect__min_df': 10, 'tfidf_vect__ngram_range': (1, 3)}
정확도: 0.8879


In [70]:
# 예측
pred_proba = grid_clf.predict_proba(X_test)[:, 1]


# 평가
auc = roc_auc_score(y_test, pred_proba)
print(f'auc: {auc:.4f}')

auc: 0.9563


## 2. 비지도학습 

In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package 

[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package mwa_ppdb to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package mwa_ppdb is already up-to-date!
[nltk_data]    | Downloading package names to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package nombank.1.0 to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nombank.1.0 is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!
[nltk_data]    | Downloading package nps_chat to
[nltk_

[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package unicode_samples is already up-to-date!
[nltk_data]    | Downloading package universal_tagset to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_tagset is already up-to-date!
[nltk_data]    | Downloading package universal_treebanks_v20 to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package universal_treebanks_v20 is already up-to-
[nltk_data]    |       date!
[nltk_data]    | Downloading package vader_lexicon to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package vader_lexicon is already up-to-date!
[nltk_data]    | Downloading package verbnet to
[nltk_data]    |     C:\Users\NEW\AppData\Roaming\nltk_data...
[nltk_data]    |   Package verbnet is already up-to-dat

True

### WordNet
- 시맨틱 정보를 포함한 어휘사전

In [2]:
from nltk.corpus import wordnet as wn

In [3]:
term = 'present'

# 용어의 시멘틱 정보를 포함한 의미, 품사 등의 정보를 가짐
synsets = wn.synsets(term)

print(type(synsets))
print(len(synsets))
print(synsets)

<class 'list'>
18
[Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [10]:
# synset 하나의 뜻

print(wn.synset('present.n.01').definition())

'the period of time that is happening now; any continuous stretch of time including the moment of speech'

In [18]:
# synset 이름
print(wn.synsets('present')[1].name())

# 뜻
print(wn.synsets('present')[1].definition())

# 품사
print(wn.synsets('present')[1].lexname())

# 부명제
print(wn.synsets('present')[1].lemma_names())

present.n.02
something presented as a gift
noun.possession
['present']


In [20]:
for synset in wn.synsets(term):
    print(f'synset 클래스: {synset.name()}')
    print(f'synset 이름: {synset.name()}')
    print(f'synset 품사: {synset.lexname()}')
    print(f'synset 의미: {synset.definition()}')
    print(f'synset 부명제: {synset.lemma_names()}')
    print()

synset 클래스: present.n.01
synset 이름: present.n.01
synset 품사: noun.time
synset 의미: the period of time that is happening now; any continuous stretch of time including the moment of speech
synset 부명제: ['present', 'nowadays']

synset 클래스: present.n.02
synset 이름: present.n.02
synset 품사: noun.possession
synset 의미: something presented as a gift
synset 부명제: ['present']

synset 클래스: present.n.03
synset 이름: present.n.03
synset 품사: noun.communication
synset 의미: a verb tense that expresses actions or states at the time of speaking
synset 부명제: ['present', 'present_tense']

synset 클래스: show.v.01
synset 이름: show.v.01
synset 품사: verb.perception
synset 의미: give an exhibition of to an interested audience
synset 부명제: ['show', 'demo', 'exhibit', 'present', 'demonstrate']

synset 클래스: present.v.02
synset 이름: present.v.02
synset 품사: verb.communication
synset 의미: bring forward and present to the mind
synset 부명제: ['present', 'represent', 'lay_out']

synset 클래스: stage.v.01
synset 이름: stage.v.01
synset 품사: verb.

In [26]:
import pandas as pd
import numpy as np

In [27]:
# 어휘 유사도 계산

tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')
rabbit = wn.synset('rabbit.n.01')

entities = [tree, lion, tiger, cat, dog, rabbit]

similarity = []
for entity in entities:
    similar = [entity.path_similarity(compared_entity) for compared_entity in entities]
    similarity.append(similar)

names = [entity.name().split('.')[0] for entity in entities]    
pd.DataFrame(np.round(similarity, 2), index = names, columns = names)

,tree,lion,tiger,cat,dog,rabbit
tree,1.00,0.07,0.07,0.08,0.12,0.08
lion,0.07,1.00,0.33,0.25,0.17,0.12
tiger,0.07,0.33,1.00,0.25,0.17,0.12
cat,0.08,0.25,0.25,1.00,0.20,0.14
dog,0.12,0.17,0.17,0.20,1.00,0.14
rabbit,0.08,0.12,0.12,0.14,0.14,1.00


### 1. SentiWordNet
- wordnet의 synset을 감성분석에 적용
- 긍정/부정/객관성 지수를 알려줌

In [29]:
import nltk
from nltk.corpus import sentiwordnet as swn

# wordnet과 다른점은 list함수를 써줘야한다는 점~
senti_synsets = list(swn.senti_synsets('slow'))

print(type(senti_synsets))
print(len(senti_synsets))
print(senti_synsets)

<class 'list'>
11
[SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [32]:
# 감성지수 출력
father = swn.senti_synset('father.n.01')
fabulous = swn.senti_synset('fabulous.a.01')

## 1. 긍정 지수
print(father.pos_score())
print(fabulous.pos_score())

## 2. 부정 지수
print(father.neg_score())
print(fabulous.neg_score())

## 3. 객관성 지수: 감성적이지 않은 단어의 경우 출력
print(father.obj_score())
print(fabulous.obj_score())

0.0
0.875
0.0
0.125
1.0
0.0


### 실습: SentiWordNet을 활용한 영화 감상평 감성 분석

In [108]:
review_df = pd.read_csv('./필요 데이터/labeledTrainData.tsv', sep = '\t')
review_df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [112]:
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

#### 텍스트 정규화_ 문서의 문장 토큰화, 단어 토큰화, 품사 태깅 사용자 함수만들기

In [73]:
from nltk.corpus import wordnet as wn

In [85]:
# senti_wordnet 형식으로 품사 바꿔주는 함수

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV

In [86]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [140]:
# 문장-> 단어토큰 -> 품사 태깅 후 senti_synset 클래스 생성하고 polarity score 합산하는 함수

def swn_polarity(text):
    lemma = WordNetLemmatizer()
    
    #문서 -> 문장 토큰화
    sentences = sent_tokenize(text)
    
    # 문장별로
    for sentence in sentences:
        senti_score = 0
        tokens_cnt = 0

        # 단어 토큰화 -> 품사 태깅
        pos_sentence = pos_tag(word_tokenize(sentence))
        
        for word, tag in pos_sentence:
            # senti_wordnet 형식의 tag로 바꿈
            wn_tag = penn_to_wn(tag)
            # 형용사, 부사, 명사인 경우만 진행
            if wn_tag not in [wn.ADJ, wn.NOUN, wn.ADV]:
                continue
            # 표제어 추출
            lemma_word = lemma.lemmatize(word, pos = wn_tag)
            # 표제어가 있는 경우만 진행
            if not lemma_word:
                continue
            
            # synset 객체 생성
            synsets = wn.synsets(lemma_word, pos = wn_tag)
            # synset 객체가 있는 경우만 진행
            if not synsets:
                continue
            
            # 감성분석
            swn_synset = swn.senti_synset(synsets[0].name())
            senti_score += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_cnt += 1

    # 감성 단어 토큰이 없는 경우 
    if not tokens_cnt:
        return 0

    # 긍정 단어 토큰이 더 많은 경우 감성 지수가 0보다 클 것
    if senti_score > 0:
        return 1
    return 0

In [141]:
text = review_df['review'][1]
swn_polarity(text)

1

In [142]:
# 사용자 함수 적용 전 확인
review_df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [143]:
# 사용자함수 적용
# 예측
review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))

# 확인
review_df.head()

,id,sentiment,review,preds
0,5814_8,1,With all this stuff going down at the moment w...,0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,0,It must be assumed that those who praised this...,1
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,1


#### 평가

In [144]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [145]:
# 평가
y_target = review_df['sentiment'].values
y_pred = review_df['preds'].values

print(f'오차행렬:\n {confusion_matrix(y_target, y_pred)}')
print(f'정확도: {accuracy_score(y_target, y_pred):.4f}')
print(f'재현율: {recall_score(y_target, y_pred):.4f}')
print(f'정밀도: {precision_score(y_target, y_pred):.4f}')
print(f'f1: {f1_score(y_target, y_pred):.4f}')

오차행렬:
 [[8351 4149]
 [6843 5657]]
정확도: 0.5603
재현율: 0.4526
정밀도: 0.5769
f1: 0.5072


### VADER

### 실습: SentiWordNet을 활용한 영화 감상평 감성 분석

In [157]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [150]:
senti_analyzer = SentimentIntensityAnalyzer()

temp_text = review_df['review'][0]

# 감성 지수 확인
senti_scores = senti_analyzer.polarity_scores(temp_text)

print(temp_text)
print(senti_scores)

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.  Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.  The actual feature film bit when it finally starts is only on for 20

In [152]:
# vader 기반 감성지수 계산 후 긍정/부정임을 나타내는 사용자함수 생성

def vader_polarity(review, threshold = 0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold를 기준으로 입력값보다 크면 1 아니면 0 반환
    agg_score = scores['compound']
    
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment    

In [153]:
review_df.head()

,id,sentiment,review,preds
0,5814_8,1,With all this stuff going down at the moment w...,0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,0,It must be assumed that those who praised this...,1
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,1


In [154]:
# 예측
review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x, 0.1))
review_df.head()

,id,sentiment,review,preds,vader_preds
0,5814_8,1,With all this stuff going down at the moment w...,0,0
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",1,1
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,0,0
3,3630_4,0,It must be assumed that those who praised this...,1,0
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,1,1


In [156]:
# 평가
y_target = review_df['sentiment'].values
y_preds = review_df['vader_preds'].values

print(f'오차행렬:\n {confusion_matrix(y_target, y_preds)}')
print(f'정확도: {accuracy_score(y_target, y_preds):.4f}')
print(f'재현율: {recall_score(y_target, y_preds):.4f}')
print(f'정밀도: {precision_score(y_target, y_preds):.4f}')
print(f'f1: {f1_score(y_target, y_preds):.4f}')

오차행렬:
 [[ 6826  5674]
 [ 1876 10624]]
정확도: 0.6980
재현율: 0.8499
정밀도: 0.6519
f1: 0.7378
